## Data Preparation and Cleaning

Carry out preprocessing of data (search for anomalies, removal / cut of outliers, etc.). Conduct time series analysis (graphical presentation, descriptive statistics).
Summarize the results of the cleaning, data preprocessing. Draw conclusions based on our analysis.


In [3]:
# Import libraries for working with tables, arrays
import pandas as pd
import numpy as np

In [5]:
sales_df = pd.read_excel("sales_train.xlsx")

items_df = pd.read_csv('items.csv', sep=';', encoding='cp1251')

stock_df = pd.read_excel('stock.xlsx')

BadZipFile: ignored

In [ ]:
items_df.head(2)

In [ ]:
sales_df.shape

In [ ]:
sales_df.shape[1]

store code, product code, brand code, shop_id, item_id, promo, brand_id, spec_promo are categorical attributes

In [ ]:
stock_df.shape[1]

To find out, of what data types items_df columns, we can use .dtypes property:

In [ ]:
items_df.info()

In [ ]:
# See all unique words in a item_category column: 
makeup_items_rus = items_df['item_category'].unique().tolist()

In [ ]:
makeup_items_rus

In [ ]:
# See all unique words in a item_category column: 
makeup_items_eng = ['makeup', 'perfume', 'grooming', 'hair_care']

In [ ]:
dict([(1, 'key'), (2, 'key1')])

In [ ]:
# Form a word-to-word dict
makeup_items_dict = dict([(makeup_items_rus[i], makeup_items_eng[i]) for i in range(len(makeup_items_rus))])

In [ ]:
makeup_items_dict

In [ ]:
# Use replace() method to replace Russian word to English ones
items_df['item_category'] = items_df['item_category'].replace(makeup_items_dict)

In [ ]:
# Check, if we replaced all items
print(f"Unique items: {items_df['item_category'].unique().tolist()}")
items_df.head(3)

In [ ]:
print(f"Dataset 'stock_df' has {stock_df.shape[0]} raws and {stock_df.shape[1]} columns")
stock_df.head(3)

Just create 2 new columns with 0 values

In [ ]:
items_df.head()

In [ ]:
sales_df[['item_category_id', 'item_category']] = 0
sales_df.head(3)

In [ ]:
#check types
items_df.dtypes

Trying to make 'item_id' column as numeric will break

In [ ]:
items_df['item_id'] = pd.to_numeric(items_df['item_id'])
# ValueError: Unable to parse string "#Н/Д" at position 535

In [ ]:
items_df[items_df['item_id']== '#Н/Д']

In [ ]:
# Filter items_df
items_df = items_df[items_df['item_id'] != '#Н/Д']
# Convert 'item_id' column to numeric, using pd.to_numeric() method
items_df['item_id'] = pd.to_numeric(items_df['item_id'])

# Check, if 'item_id' column became numeric
items_df.dtypes

In [ ]:
from tqdm import tqdm
for i in tqdm(range(10000)):
    ...

In [ ]:
items_df.shape[0]

In [ ]:
items_df.iloc[1360]

In [ ]:
!pip install tqdm

In [ ]:
from tqdm.notebook import tqdm


# Extract the corresponding item_id to item_category_id
items_df_dict_id = dict([(items_df.iloc[i]['item_id'], 
                          items_df.iloc[i]['item_category_id']) 
                            for i in range(items_df.shape[0])])

In [ ]:
items_df_dict_id

In [ ]:
# Extract the corresponding item_id to item_category
items_df_dict_category = dict([(items_df.iloc[i]['item_id'], 
                                items_df.iloc[i]['item_category']) 
                                    for i in range(items_df.shape[0])])

In [ ]:
items_df_dict_category

In [ ]:

# print(items_df_dict_category)  # {20224176006: 'makeup',  ... etc.}

item_category_id, item_category = [], []

In [ ]:
tqdm(sales_df['item_id'].tolist())

In [ ]:
for item_id in tqdm(sales_df['item_id'].tolist()):
    
    if item_id in items_df_dict_id.keys():  # Check if category in a dict keys
        item_category_id.append(items_df_dict_id[item_id])

    else:
        item_category_id.append(0)  # For unknown category set 0

    if item_id in items_df_dict_category.keys():  # Check if category in a dict keys
        item_category.append(items_df_dict_category[item_id])

    else:
        item_category.append(0)  # For unknown category set 0

In [ ]:
# Input data into table
sales_df['item_category_id'] = item_category_id
sales_df['item_category'] = item_category

In [ ]:
# Group the table by item_category_id
# See that for 406 items we don't know the category type:
sales_df.groupby('item_category_id').count()

So, we don't need items_df anymore theese information was added into sales_train_df in the last ones two columns

In [ ]:
# rename sales_train_df to a simple df
df = sales_df
df.head(3)

In [ ]:
stock_df.head(3)

Days with 0 balance unique values

In [ ]:
stock_df['days with 0 balance'].unique()

In [ ]:
stock_df.dtypes

In [ ]:
type(df.iloc[1]['item_id'])

Add data 'days with 0 balance' from stock_df to df dataframe

In [ ]:
# Create free list for days saving
days = []

In [ ]:
# This code works very slow - it's because of python for cycle,
# not pandas internal functions, or numpy arrays - using them are
# preferable
for i in tqdm(range(df.shape[0])):  # tqdm shows the processing of cycle
    # Filter elements
    store = df.iloc[i]['shop_id']
    month = df.iloc[i]['Month']
    year = df.iloc[i]['Year']
    itemid = df.iloc[i]['item_id']
    # Take value of 'days with 0 balance' column
    day = stock_df['days with 0 balance'][  # filter values
                    (stock_df['store'] == store) &
                    (stock_df['month'] == month) &
                    (stock_df['year'] == year) &
                    (stock_df['itemid'] == itemid)]
    # Check if day exists
    if len(day) != 0:
        # print(day.values[0])
        days.append(day.values[0])
    else:
        # We believe that there were no days 
        # when the product was not sold
        days.append(0)

In [ ]:
# Add new column to df dataframe with name 'days_with_0_balance':
df['days_with_0_balance'] = days
df.head(2)

In [ ]:
df['days_with_0_balance'].unique()

It is not very clear, but maybe it's not a bad one idea to add a new column - number of days, last from the beggining of the year

In [ ]:
def days(date, year_num):
    # Function to make a calculus for a day according to 
    return (date // 1000000000 - 1514764800) // 86400 + 1 - 365 * year_num

In [ ]:
# Count days for every row in a df
days_2018 = [days(date, 0) for date in df['date'][df['Year']  == 2018].values.tolist()]
days_2019 = [days(date, 1) for date in df['date'][df['Year']  == 2019].values.tolist()]
days_2020 = [days(date, 2) for date in df['date'][df['Year']  == 2020].values.tolist()]

In [ ]:
# Concatenate all days values
df['days_from_beginning'] = days_2018 + days_2019 + days_2020
df.head(5)

Analyze discamount column - there are some values, that are below zero

In [ ]:
print(df['discamount'][df['discamount'] < 0].unique())

In [ ]:
df['discamount'][df['discamount'] < 0].count().
# There are only 58 values that are below zero

So, we can see, that a lot of values are below zero, but most of it is more than zero

In [ ]:
# Count a price 
df['sales'] = df['item_cnt_day'] * df['item_price']

# Count a price with discamount
df['discamount sales'] = df['item_cnt_day'] * df['item_price'] - df['discamount']

# Show first 3 rows of dataframe
df.head(3)

## Exploratory Analysis and Visualization

In a previous part we created a new dataframe, that include all the data in given files. We also added some new columns, like 'sales' column.
In this part let's see some statistics, find out sum of discamount sales, according to years in a pieplot, graph histogram graphs and plot an correlation matrix to see all the values dependences in a dataframe.

Let's begin by importing`matplotlib.pyplot` and `seaborn`.

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (9, 5)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

Compute the mean, sum, range and other interesting statistics for numeric columns

In [ ]:
# Extract 'item_price' data
pd_item_price = df['item_price']
np_item_price = np.array(df['item_price'].tolist())

# Explore mean values of prices
print(f'pandas mean function: {pd_item_price.mean()}')
print(f'numpy mean function: {np_item_price.mean()}')

# Explore min values of prices
print(f'pandas min function: {pd_item_price.min()}')
print(f'numpy min function: {np_item_price.min()}')

# Explore max values of prices
print(f'pandas max function: {pd_item_price.max()}')
print(f'numpy max function: {np_item_price.max()}')

Actually, we have great pandas function .describe() to make some basic statistics

In [ ]:
df.describe()

**TODO** - Explore one or more columns by plotting a graph below, and add some explanation about it

In [ ]:
df_pie = df.groupby('Year')[['discamount sales', 
                             'days_with_0_balance']].sum()

# Make the plot with pandas
df_pie.plot(kind='pie', subplots=True, figsize=(12, 8))
plt.title("Summ of discamount sales, according to Years")
plt.show()

We can say that total sales in 2019 more then in a 2018. According to 'days_with_0_balance' it can be say that in 2019 there were more days with zero balance

In [ ]:
# Form a dataframe using .groupby() method
df_pie_shop_id = df.groupby('shop_id')[['discamount sales', 
                                'days_with_0_balance']].sum()
# A groupby() pandas method is used when we need to group
# large amounts of data and compute some operations on these groups


# Make the plot with pandas
df_pie_shop_id.plot(kind='pie', subplots=True, figsize=(12, 8))
plt.title("Sum of 'discamount sales' and 'days_with_0_balance', according to shop_id")
plt.show()

So, 'discamount sales' are almost equals for all stores

**TODO** - Explore one or more columns by plotting a graph below, and add some explanation about it

Let's look at the values of Item_cnt_day – the number of sales in pieces. 20 unique values:

In [ ]:
df['item_cnt_day'].unique().tolist()

The data contains negative values - apparently, these are product returns, since the number of sales is a non - negative value.

In [ ]:
df['item_cnt_day'][df['brand_id'] == 10013].hist(figsize=(8, 8));

So, we can see that most part of days items did not sale in a 10013 store

**TODO** - Explore one or more columns by plotting a graph below, and add some explanation about it

Plot the barplot graph for brand 10013 for 'Month'

In [ ]:
df['Month'][df['brand_id'] == 10013].hist(figsize=(8, 8));

So, least of all sales in a 'July' month, and the most sales are in the winter. The period of occurrence of seasonal fluctuations is more often predicted, and it is influenced by:


*   New year, Christmas, March 8, may and other holidays.
*   In summer, demand is minimal, which is associated with the peak of vacations;
*   Average age, characteristics of the target audience.

**TODO** - Explore one or more columns by plotting a graph below, and add some explanation about it

In [ ]:
# Explore relationship between columns using scatter plots, bar charts etc.

sns.boxplot(data=df, x='item_category_id', y="promo_time");

Here's what the boxplot shows:

The Median is the value of the element in the center of the ranked row.
For example, if you put all octopuses in ascending order of their ratings, the median will be the rating that the octopus put in the middle. This means that half of the octopuses on the right rated the probability of buying lower, and the other half (on the left) higher than the median.

The upper quartile is a score above which only 25% of the scores are higher.
The lower quartile is a value below which only 25% of estimates are made.
The interquartile range (IQR) is the difference between 75% and 25% quartile. Within this range lies 50% of observations. If the range is narrow (as in the case of octopuses), then the members of the subgroup are unanimous in their assessments. If it is wide, it means that there is no uniform opinion (as in chickens).

Outliers are atypical observations. What exactly is considered atypical? Those estimates that go beyond:

values of 25% percentile minus 1.5 x IQR
values of 75% percentile plus 1.5 x IQR

From the boxplot it is clear, that promo time depends on 'item_category_id'.

In [ ]:
df[["promo_time"]].plot(kind='box');

**TODO** - Explore one or more columns by plotting a graph below, and add some explanation about it

In [ ]:
# Use the built-in corr method()
corr_matrix = df.corr()

Let's look at the Pearson correlation matrix, which is a measure of the degree and positivity of linear relationships between two variables. A value of +1 indicates a direct proportionality between the feature values. If the value is -1, on the contrary, there is a correlation, but with a negative coefficient:

In [ ]:
# Pearson graph parameters
plt.figure(figsize=(20,14), dpi=60)
sns.heatmap(corr_matrix, 
            xticklabels=corr_matrix.columns, 
            yticklabels=corr_matrix.columns, 
            cmap='RdYlGn', 
            center=0, 
            annot=True)

# Specify additional parameters (the size of the text, the title, and the title itself)
plt.title('The correlation matrix for the parameters data table', fontsize=20)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show();

Of course, it would be more correct to build a heatmap for each pair (store, product), which shows how many units of the product were sold during the entire time.

From the correlation table, we can see that the duration of the promo_time discount depends quite strongly on the size of the size_disc (0.61) discount. Also see that having a discount promo (0.71), the value of the discount size_disc (0.72), and duration of action discounts promo_time (0.46) depend on the size of the discount discamount, and is, in principle, so understood that they are interrelated, although the dependence of the duration of the sale size is not obvious. There is also a good correlation of the duration of the promotion from the month (0.58) and slightly less from the day of the week (0.29). This may be due to holidays (new year's), weekends/Fridays, or the holiday period (less people started buying certain products) or seasonality (tanning products are unlikely to be actively bought in the winter, except for those who are going on vacation at this time). Of the negative correlations, the most notable is the dependence of the availability of the promotion on the promo product with a special discount spec_promo (-0.24), that is, the product participating in the simple action cannot participate in the promo action.

Let us save and upload our work to Jovian before continuing

## Asking and Answering Questions

TODO - write some explanation here.



#### Q1: What was the best month for sales in 2018? How much money was earned that month?

In [ ]:
# What was the best month for sales in 2018? 
# How much money was earned that month?

df_2018 = df[df['Year'] == 2018].copy()
best_months_2018 = df_2018.groupby('Month')['sales'].sum()
best_months_2018

In [ ]:
print(f"Max profit was on {best_months_2018.index.max()}s moth - it's December")
print(f"And the profit was {best_months_2018.max()} roubles, that is about {best_months_2018.max() // 90}$ in December, 2018")

In [ ]:
# make values for month using numpy arange() method
months = np.arange(1, 13, 1)
print(months)

plt.bar(months, best_months_2018.tolist());

#### Q2: What are new products wich were added in 2019 year? What are products id's and the number of products that are no longer available in 2019

In [ ]:
# List unique goods items in 2018 and 2019 years
items_count_in_2018 = set(df['item_id'][df['Year'] == 2018].unique().tolist())
items_count_in_2019 = set(df['item_id'][df['Year'] == 2019].unique().tolist())

In [ ]:
# What are new products wich were added in 2019 year?
print(f"In 2019 were added {len(items_count_in_2019 - items_count_in_2018)} new products")
print(f"Theese are products with id: {items_count_in_2019 - items_count_in_2018}")

In [ ]:
# What are products id's and the number of products that are no longer available in 2019?
print(f"The number of products that are no longer available in 2019 is {len(items_count_in_2018 - items_count_in_2019)} pieces")
print(f"these are products with id: {items_count_in_2018 - items_count_in_2019}")

#### Q3: What is the best day of week for sales?

In [ ]:
df['date_block_num'].unique()

In [ ]:
# First, we should convert 'date' column to datetime using .to_datetime() method
# Agter that is done, we can get day of week by .dt.dayofweek:
df['day_of_week'] = pd.to_datetime(df['date']).dt.dayofweek
df['day_of_week'].head(3)

In [ ]:
df['day_of_week'].unique()

In [ ]:
day_of_week_sales = df.groupby('day_of_week')['sales'].sum()
sales_dict = dict([(sales, i) for i, sales in enumerate(day_of_week_sales)])
sales_dict

In [ ]:
max_day_sales = df.groupby('day_of_week')['sales'].sum().max()

print(f"Max sales are {max_day_sales // 90} $ for all time")
print(f"in a {sales_dict[max_day_sales]} day of week")

So, saturday is the best day for sales.

#### Q4: What is the most popular item and what category is it?

In [ ]:
# So, to find that out, we should check 'item_cnt_day' column
# and count sum of it, according to 'item_id'
most_popular_item = df.groupby(['item_category', 'item_id'])['item_cnt_day'].sum()

In [ ]:
category, item = most_popular_item.idxmax()
category, item

In [ ]:
print(f"Most popular item was {item} of {category} category")
print(f"This item was sold {most_popular_item.max()} times")

#### Q5: What brand is the most popular? 

In [ ]:
# Form a dataframe using .groupby() method
df_pie_shop_id = df.groupby('brand_id')['item_cnt_day'].sum()
# A groupby() pandas method is used when we need to group
# large amounts of data and compute some operations on these groups

# There are only 2 brands in data
# Make the plot with pandas
df_pie_shop_id.plot(kind='pie', subplots=True, figsize=(12, 8))
plt.title("Distribution of the brand's bestselling")
plt.show()

Brand with id '10021' is more popular, than '10013'.

#### Q6: What are counts of returned items?

In [ ]:
print('Total negative values: ', end='')
print(df[df ['item_cnt_day'] < 0]. item_cnt_day.count())
print('Of them equal to -1: ', end='')
print(df[df['item_cnt_day'] == -1].item_cnt_day.count())
print('Of them equal to the value of -2: ', end='')
print(df[df['item_cnt_day'] == -2].item_cnt_day.count())
print('Of them equal to the value -3: ', end='')
print(df[df['item_cnt_day'] == -3].item_cnt_day.count())
print('Of them equal to the value -4: ', end='')
print(df[df['item_cnt_day'] == -4].item_cnt_day.count())

In other words, in most cases, refunds are single.

## Inferences and Conclusion

**TODO** - Write some explanation here: a summary of all the inferences drawn from the analysis, and any conclusions you may have drawn by answering various questions.

First of all, it is worth noting that there is not as much data as we would like to build a high-quality sales forecasting model.
For more accurate and effective sales forecasting, the current data set needs to be enriched with data on the number of days of work in the location - the store may have been under repair, for example, or in quarantine. Bad sales of a location can also be affected by the presence of strong competitors - and sales are always bad there. These, and many other factors, would probably be useful.

You can notice some features of the presented data set: for example, the average duration of promotions (Promo_time) is 5 days (4.801320), and the maximum is about 9 days.
The discount amount varies greatly - from 0 to 50%.

According to distribution by discount values, we can say that most of the values are the absence of a discount, that is, most of the time products are displayed at the usual price, and if a discount is valid, it is usually 50% or 40%.

First of all, we downloaded hand-made by myself Kaggle dataset. Then loaded it into pandas datafraames, and started to concatenate files into one dataframe. After that work is done, we added new columns into dataframe to make it mor consolidated and informative. Finally, we got some statistical inforamtion, answered some interesting questions about data, and plot graphs. In the end, we made conclusions about this work.